# Try the igraph methods for Girvan-Newman and Newman's fast algorithm

- Created a wrapper for running the methods
    - had to do a workaround because of a bug
    
    
- Created a wrapper to test methods on GN synthetic test graphs
 
---

In [1]:
import igraph 
import networkx as nx

import sys
sys.path.append('../my_modules/')
from girwan_newman_benchmark import create_GN_benchmark_graph,fraction_of_vertices_correctly_classified

#### Define functions for running a clustering


In [2]:
def best_modularity_level_w_igraph_method(graph,igraph_method):
    """
    Run an igraph clustering method on the graph, return best clustering.
    
    Run the method, and select the best level of the dendogram.
        igraph select the best level based on hint from the algorithm
        (fastgreedy calculates modularities for each level on the fly ) or 
        if there is no hint recalculates modularity for each level, and selects the best one.
    Do workaround to avoid bug with initally disconnected graphs.
    """
    
    #run clustering
    dendrogram=igraph_method(graph)
    
    """
    Workaround:
        usually you would call dedrogram.as_clustering() it should return the best
        level of the dendrogram.
        BUT! there is a bug for an initially  unconnected graph, where this causes this method to fail.
            https://github.com/igraph/igraph/issues/675
    The workaround is to try to get the best cut, but if fails, get a lower one which at least exists.
    """
    
    #get best cut:
    n_best_clusters=dendrogram.optimal_count
    
    #get best cluster membership
    while(n_best_clusters <100):
        #try with the best, if it fails increment level
        try:
            best_clusters=dendrogram.as_clustering(n_best_clusters).membership
            break
        except:
            n_best_clusters+=1
    
    return best_clusters

#### Define functions to evaluate a method

In [3]:
def nx_2_ig(nx_g):
    """
    Covert networkx graph to igraph graph.
    
    I started to work in networkx because i have already worked with that
    but later i started to use igraph because it has the fastgreedy
    algorithm implemented, and it is also supposed to be faster.
    """
    
    ig_g=igraph.Graph()
    #igraph docs say, you can add edges by name
    # but you actually cannot ...
    # so i maintain a name - id mapping
    id_name,name_id=dict(),dict()
    for node,i in zip(nx_g.nodes(),range(128)):
        ig_g.add_vertex(name=node)
        id_name[i],name_id[node]=node,i
    
    for edge in nx_g.edges():
        ig_g.add_edge(name_id[edge[0]],name_id[edge[1]])
 
    return ig_g,id_name,name_id

def test_igraph_method(k_in,igraph_method):
    """
    Test an igraph community detection method on a random synthetic GN benchmark graph, return fvcc score.
    
    fvcc score is the fraction_of vertices correctly classified defined by Newman.
    """
    
    #create graph
    graph = create_GN_benchmark_graph(k_in=k_in)
    #turn it into an igraph graph
    ig_g,id_name,name_id = nx_2_ig(graph)
    
    # run the method with best level
    membership =  best_modularity_level_w_igraph_method(ig_g,igraph_method)
    
    #calculate quality of clustering
    fvcc=fraction_of_vertices_correctly_classified(membership)
    
    return fvcc

#### Run test

In [4]:
GN=igraph.Graph.community_edge_betweenness
print test_igraph_method(k_in=16,igraph_method=GN)

1.0


In [5]:
N_fast=igraph.Graph.community_fastgreedy
print test_igraph_method(k_in=9,igraph_method=N_fast)

0.890625
